In [1]:
%matplotlib inline

import numpy as np
from collections import OrderedDict, Counter, defaultdict
import pandas as pd

import Bio
from Bio import SeqIO

import seaborn as sns
import matplotlib.pyplot as plt

import glob

import subprocess
from subprocess import call

import re

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas


Bad key text.latex.unicode in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
https://githu

# Notebook plan

* extraction of extension sequences and transcriptomic (local) coordinates for gencode 25 .fasta files
---
* local coordinates of 50-codons-long parts of extensions 
---
* global coordinates of extensions/50-codons-long parts 
---
* overlaps of 50-codons with coding exons: gencode25/35/refseq, overlaps frame comparison (separate notebooks)
---
* add PhyloCSF 120-mammals pipeline (separate scripts most likely)
---
* save to df with local/global coo/PhyloCSF score/overlaps. Next step is to add Proteomics/Ribo-seq data in separate notebook 

# Create Metadata dataframe for gencode 25/35 releases

In [24]:
def extrac_metadata(filepath, pc=True):
    if pc==True:
        troubles = []
        li = []
        with open(filepath, "r") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                record_id = str(record.id)
                tr_id = record_id.split('|')[0]
                seq = str(record.seq)
                
                gene = record_id.split('|')[5]
                gene_tr = record_id.split('|')[4]
                
                # CDS extraction
                m_cds = re.finditer(r'(CDS:)[0-9]*-[0-9]*', record_id)
                cds = [x.group(0) for x in m_cds][0] #UTR3:955-1054
                cds_start = int(cds.split(':')[1].split('-')[0]) - 1
                cds_end = int(cds.split(':')[1].split('-')[1]) 
                cds_seq = seq[cds_start:cds_end]     
                cds_start_codon = cds_seq[0:3]
                cds_stop_codon = cds_seq[-3:]
        
                
                # UTR5 extraction
                # some transcript do not have UTR5 annotated 
                # but we still want to save such trascripts
                m_utr5 = re.finditer(r'(UTR5:)[0-9]*-[0-9]*', record_id)
                t = [x.group(0) for x in m_utr5]
                
                
                if t:
                    utr5 = t[0] #UTR5:1-1054
                    utr5_start = int(utr5.split(':')[1].split('-')[0]) - 1
                    utr5_end = int(utr5.split(':')[1].split('-')[1]) 
        
                    if utr5_start == utr5_end: #1=1
                        print ('utr5_start == utr5_end', record_id)
                        utr5_start_seq = seq[utr5_start-1:(utr5_end)]
                    else: 
                        utr5_start_seq = seq[utr5_start:(utr5_end)]
                        
                    li.append([tr_id, gene, gene_tr, seq, utr5_start_seq, 
                   utr5_start_seq[-3:], cds_seq,
                  cds_start_codon, cds_stop_codon, cds_start, cds_end,
                  utr5_start, utr5_end, record_id])
                    
                    
                else:
                    utr5_start = 0
                    utr5_end = 0
                    utr5_start_seq = None
                    cds_start = 0
                    
                    li.append([tr_id, gene, gene_tr, seq, utr5_start_seq, 
                   cds_seq[:3], cds_seq,
                  cds_start_codon, cds_stop_codon, cds_start, cds_end,
                  utr5_start, utr5_end, record_id])
        
        metadata_pc_g25 = pd.DataFrame(li, columns = ['tr_id', 'gene' ,'gene_tr', 
                                 'transcript_seq', '5UTR_start_seq', 
                                 'CDS_start', 'cds_seq', 'cds_start_codon', 
                                 'cds_stop_codon', 'cds_start_pos',  'cds_stop_pos',
                                 'utr5_start', 'utr5_end', 'record_id'])
        return (metadata_pc_g25)

In [25]:
metadata_pc_g25 = extrac_metadata(filepath="data/gencode.v25.pc_transcripts.fa", pc=True)
print ('# trs with 5UTR:', metadata_pc_g25[metadata_pc_g25['5UTR_start_seq'].notna()].shape[0], '; # all trs', metadata_pc_g25.shape[0])
metadata_pc_g25.to_csv('tmp_res/metadata_pc_g25.txt', sep='\t', index=False)

metadata_pc_g25[0:2]

# trs with 5UTR: 78873 ; # all trs 94359


,tr_id,gene,gene_tr,transcript_seq,5UTR_start_seq,CDS_start,cds_seq,cds_start_codon,cds_stop_codon,cds_start_pos,cds_stop_pos,utr5_start,utr5_end,record_id
0,ENST00000335137.3,OR4F5,OR4F5-001,ATGGTGACTGAATTCATTTTTCTGGGTCTCTCTGATTCTCAGGAAC...,None,ATG,ATGGTGACTGAATTCATTTTTCTGGGTCTCTCTGATTCTCAGGAAC...,ATG,TAG,0,918,0,0,ENST00000335137.3|ENSG00000186092.4|OTTHUMG000...
1,ENST00000624431.1,FO538757.2,FO538757.2-201,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,AAG,ATGAGTGACAGCATCAACTTCTCTCACAACCTAGGCCAGCTCCTGT...,ATG,TAG,316,718,0,316,ENST00000624431.1|ENSG00000279928.1|-|-|FO5387...


In [26]:
metadata_pc_g35 = extrac_metadata(filepath="data/gencode.v35.pc_transcripts.fa", pc=True)
print ('# trs with 5UTR:', metadata_pc_g35[metadata_pc_g35['5UTR_start_seq'].notna()].shape[0], '; # all trs', metadata_pc_g35.shape[0])
metadata_pc_g35.to_csv('tmp_res/metadata_pc_g35.txt', sep='\t', index=False)

metadata_pc_g35[0:2]

# trs with 5UTR: 85118 ; # all trs 101486


,tr_id,gene,gene_tr,transcript_seq,5UTR_start_seq,CDS_start,cds_seq,cds_start_codon,cds_stop_codon,cds_start_pos,cds_stop_pos,utr5_start,utr5_end,record_id
0,ENST00000641515.2,OR4F5,OR4F5-202,CCCAGATCTCTTCAGTTTTTATGCCTCATTCTGTGAAAATTGCTGT...,CCCAGATCTCTTCAGTTTTTATGCCTCATTCTGTGAAAATTGCTGT...,GTT,ATGAAGAAGGTAACTGCAGAGGCTATTTCCTGGAATGAATCAACGA...,ATG,TAG,60,1041,0,60,ENST00000641515.2|ENSG00000186092.6|OTTHUMG000...
1,ENST00000335137.4,OR4F5,OR4F5-201,TCCTGGAATGAATCAACGAGTGAAACGAATAACTCTATGGTGACTG...,TCCTGGAATGAATCAACGAGTGAAACGAATAACTCT,TCT,ATGGTGACTGAATTCATTTTTCTGGGTCTCTCTGATTCTCAGGAAC...,ATG,TAG,36,954,0,36,ENST00000335137.4|ENSG00000186092.6|OTTHUMG000...


# local coordinates of N-terminal-Extensions 

In [27]:
# extracting full NTEs

def get_pos_of_1st_stop_inframe(seq):
    '''
    seq = seq of 5'leader
    '''
    
    # reverse seq
    seq_r = seq[::-1]
    
    i = 0  
    li = []
    while i <= len(seq_r):    
    
        codon = seq_r[i:i+3][::-1] # read by codon and reverse it like it's read on forward strand
        if (codon == 'TAG') | (codon == 'TGA') | (codon == 'TAA'):
            li.append(i) 
        i = i + 3
        
    if li:
        pos = (np.min(li))
        pos_f = len(seq) - pos
    else:
        pos_f = 0
    
    return (pos_f)


def cut_nt(x):
    
    if x['len'] % 3 == 0:
        x['N_term_start1'] = x['N_term_start']
        x['N_term_end1'] = x['N_term_end']
        
    elif x['len'] %3 == 1:
        x['N_term_start1'] = x['N_term_start'] + 1
        x['N_term_end1'] = x['N_term_end']
        
        
    elif x['len'] %3 == 2:
        x['N_term_start1'] = x['N_term_start'] + 2
        x['N_term_end1'] = x['N_term_end']
        
    else:
        pass
        
    return x    

def local_coo_df(metadata):
    new_li = []
    
    for row in metadata.dropna().to_numpy():
        tr_seq = row[3]
        cds_start = row[9]
        cds_end = row[10]
        utr5_start = int(row[11])
        utr5_end = int(row[12])
        tr_id = row[0]
        gene = row[1]

        seq = tr_seq[utr5_start:utr5_end]
        pos_stop = get_pos_of_1st_stop_inframe(seq)
    
        if (pos_stop is not None):
            if (pos_stop != cds_start):
                pos_stop_real = pos_stop + utr5_start
                # transcriptomic pos of N-terminal extension
                tr_pos = [pos_stop_real, cds_start]
                new_li.append([tr_id, pos_stop_real, cds_start])
        
        else:
            tr_pos = [utr5_start+0, cds_start]
            new_li.append([tr_id, utr5_start+0, cds_start])
        
    local_coo = pd.DataFrame(new_li, columns = ['tr_id', 'N_term_start', 'N_term_end'])
    local_coo['len'] = local_coo['N_term_end'] - local_coo['N_term_start'] 
    
    # check frame: NTE should have length multiple of 3nt = codon
    
    local_coo = local_coo.apply(cut_nt, axis=1)
    local_coo['len_codons'] = (local_coo['N_term_end1'] - local_coo['N_term_start1']) / 3 
    local_coo = local_coo.merge(metadata[['tr_id', 'gene']], on='tr_id', how='inner')
    local_coo.to_csv('tmp_res/NTE_local_coordinates.txt', sep='\t', index=False)
    
    return (local_coo)

local_coo = local_coo_df(metadata=metadata_pc_g25)
local_coo.shape

(77820, 8)

In [28]:
# why local_coo.shape = 77820, and 78873 trs in total have UTR5

set_of_closest_to_CDS_codons = set()

for el in metadata_pc_g25[(metadata_pc_g25['5UTR_start_seq'].notna()) & (metadata_pc_g25['tr_id'].isin(list(set(metadata_pc_g25['tr_id'].tolist()) - set(local_coo['tr_id'].tolist()))))].to_numpy():
    tr = el[0]
    utr5_seq = el[4]
    
    set_of_closest_to_CDS_codons.add(utr5_seq[-3:])

# stop codons right before CDS => no NTEs are found 
set_of_closest_to_CDS_codons

{'TAA', 'TAG', 'TGA'}

# Local cordinates for 50-codons-long parts of extensions 

In [29]:
def add_new_NTE_start_50_codons(x):
    '''
    return N_term_start_50
    if there are >50 codons = all 50
    if less = all we have 
    '''
    if x['len_codons'] >= 50:
        New_nte_start = x['N_term_end1'] - 150  
    else:
        New_nte_start = x['N_term_start1']
        
    return New_nte_start
        

local_coo['50len_flag'] = local_coo.apply(add_new_NTE_start_50_codons, axis=1)

In [30]:
local_coo[['tr_id', 'N_term_end1', '50len_flag']].to_csv('tmp_res/local_coo_50codons_and_less.txt', sep='\t', index=False)
local_coo[['tr_id', 'N_term_start1', 'N_term_end1']].to_csv('tmp_res/local_coo_primary_NTE.txt', sep='\t', index=False)

# Global coordinates 

Separate R-script is used to extract genomic coordinates using .gtf files, GenomicFeatures library: transcriptomic_to_genomic_coordunates_example.R


In [1]:
def prepare_global_coo(path_to_file_with_global_coo, colname):
    global_coo_g25 = pd.read_csv(path_to_file_with_global_coo, sep='\t')
    global_coo_g25_f = global_coo_g25[global_coo_g25['hit'] == True]
    global_coo_g25_f = global_coo_g25_f.sort_values(by=['seqnames', 'start'])
    
    global_coo_g25_f['global_coo'] = global_coo_g25_f['seqnames']+':'+global_coo_g25_f['start'].astype(str) +'-'+global_coo_g25_f['end'].astype(str)
    global_coo_g25_f = global_coo_g25_f[['group_name', 'strand', 'global_coo']].groupby(['group_name', 'strand']).agg('+'.join).reset_index()
    global_coo_g25_f.columns = ['tr_id', 'strand', colname]
    return global_coo_g25_f

In [2]:
primary_global = prepare_global_coo(path_to_file_with_global_coo='tmp_res/primary_NTE_global_coo.txt',
                                   colname = 'global_coo_primary')

codons50_global = prepare_global_coo(path_to_file_with_global_coo='tmp_res/global_coo_50codons_and_less.txt',
                                    colname = 'global_coo_50_and_less')

print ('tr # check:',codons50_global.tr_id.nunique(), primary_global.tr_id.nunique(), local_coo.tr_id.nunique())

NameError: name 'pd' is not defined

In [33]:
# add global coo to local df

local_and_global_df = local_coo.merge(codons50_global[['global_coo_50_and_less', 'tr_id']], 
                                      on='tr_id', how='inner').merge(primary_global, on='tr_id', how='inner').merge(metadata_pc_g25[['tr_id', 'transcript_seq', '5UTR_start_seq']], on='tr_id', how='inner')


print (local_and_global_df.shape, local_and_global_df.tr_id.nunique())

local_and_global_df[0:3]

(77820, 14) 77820


,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,strand,global_coo_primary,transcript_seq,5UTR_start_seq
0,ENST00000624431.1,274,316,42,274,316,14.0,FO538757.2,274,chr1:182667-182708,+,chr1:182667-182708,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...
1,ENST00000624735.1,0,236,236,2,236,78.0,FO538757.1,86,chr1:200087-200236,-,chr1:200087-200320,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...
2,ENST00000420190.6,60,504,444,60,504,148.0,SAMD11,354,chr1:924282-924431,+,chr1:923988-924431,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...


In [34]:
local_and_global_df.to_csv('tmp_res/local_and_global_coo_50_and_full.txt', sep='\t', index=False)

# Exclude cases with coding exons overlaps 

In [35]:
def get_bed(filepath, df):
    '''
    df: tr_id, strand, coo
    '''
    f = open(filepath, 'w')

    for el in df.to_numpy():
        coo = el[2].split('+')
        strand = el[1]
        tr_id = el[0]
    
        for coo_ in coo:
            chrom = coo_.split(':')[0]
            start = coo_.split(':')[1].split('-')[0]
            stop = coo_.split(':')[1].split('-')[1]
        
        
            if int(start) - int(stop) == 1:
                f.write(chrom+'\t'+start+'\t'+str(int(stop)+1)+'\t'+tr_id+'\t1\t'+strand+'\n') 
                # where the length of the region is 0 nt
            else:
                f.write(chrom+'\t'+start+'\t'+stop+'\t'+tr_id+'\t1\t'+strand+'\n')
            
    f.close()  

make .bed file of 50-codons-long parts of extensions 

In [36]:
get_bed(filepath='tmp_res/50_codons_NTE.bed', df=local_and_global_df[['tr_id', 'strand', 'global_coo_50_and_less']])

make .bed file of Full parts of extensions 

In [37]:
get_bed(filepath='tmp_res/full_NTE.bed', df=local_and_global_df[['tr_id', 'strand', 'global_coo_primary']])

## Gencode25

calculate global coordinates for coding exons in R script using metadata_pc_g25 file as supply of local coordinates of CDS 

In [38]:
cod_ex_25 = pd.read_csv('tmp_res/coding_exons_g25_global.txt', sep='\t')
cod_ex_25 = cod_ex_25[cod_ex_25['hit'] == True]
cod_ex_25[['seqnames', 'start', 'end', 'group_name', 'exon_rank', 'strand']].to_csv('tmp_res/coding_exons_g25_global.bed', sep='\t', index=False, header=None)

* correct starts 

In [39]:
cod_ex_25 = pd.read_csv('tmp_res/coding_exons_g25_global.bed', sep='\t', header=None)
cod_ex_25.columns = ['chr_cod', 'start_cod', 'end_cod', 'tr_id_cod', 'exon_cod', 'strand_cod']
cod_ex_25['start_cod'] = cod_ex_25['start_cod'] - 1 #correct start now in GWIPs

cod_ex_25.to_csv('tmp_res/cod_ex_25_corrected.bed', sep='\t', index=False, header=None)
cod_ex_25[0:2]

,chr_cod,start_cod,end_cod,tr_id_cod,exon_cod,strand_cod
0,chr1,69090,70008,ENST00000335137.3,1,+
1,chr1,182708,182746,ENST00000624431.1,1,+


## Gencode 35

In [40]:
cod_ex_35 = pd.read_csv('tmp_res/coding_exons_g35_global.txt', sep='\t')
cod_ex_35 = cod_ex_35[cod_ex_35['hit'] == True]
cod_ex_35[['seqnames', 'start', 'end', 'group_name', 'exon_rank', 'strand']].to_csv('tmp_res/coding_exons_g35_global.bed', sep='\t', index=False, header=None)

* correct starts 

In [41]:
cod_ex_35 = pd.read_csv('tmp_res/coding_exons_g35_global.bed', sep='\t', header=None)
cod_ex_35.columns = ['chr_cod', 'start_cod', 'end_cod', 'tr_id_cod', 'exon_cod', 'strand_cod']
cod_ex_35['start_cod'] = cod_ex_35['start_cod'] - 1 #correct start now in GWIPs

cod_ex_35.to_csv('tmp_res/cod_ex_35_corrected.bed', sep='\t', index=False, header=None)
cod_ex_35[0:2]

,chr_cod,start_cod,end_cod,tr_id_cod,exon_cod,strand_cod
0,chr1,65564,65573,ENST00000641515.2,2,+
1,chr1,69036,70008,ENST00000641515.2,3,+


## Refseq

- one option - use latest .gff file and parse it, extract CDS coordinates (5May, https://www.ncbi.nlm.nih.gov/projects/genome/guide/human/index.shtml) 
- another option - use UCSC Table Browser (I was not sure that it provides the latest version, so I used 1st option)

- GRCh38_RefSeq2UCSC.txt: chromosome ids conversion file, https://github.com/dpryan79/ChromosomeMappings/blob/master/GRCh38_RefSeq2UCSC.txt 

In [42]:
import sys                                                                         
from gffutils.iterators import DataIterator                                        

li = []

# extract gene and 
                                                                                
for feature in DataIterator('data/GRCh38_latest_genomic.gff'):                                                                                                 
    if feature.featuretype == "CDS":   
        #print (feature.seqid, feature.featuretype, len(feature), feature.chrom, feature.start, feature.stop, feature.end, feature.strand)
        try:
            #print (feature.attributes['Parent'])
            li.append([feature.attributes['Parent'][0].split('rna-')[1], feature.attributes['gene'][0], feature.seqid, feature.featuretype, feature.chrom, feature.start, feature.stop, feature.end, feature.strand])
        except Exception as e:
            print (e)
df = pd.DataFrame(li, columns = ['tr_id', 'gene', 'seqid', 'type', 'chrom', 'start', 'stop', 'end', 'strand'])
df[0:4]


print (set([x.split('_')[0] for x in df['tr_id'].tolist()]))

chr_map = pd.read_csv('data/GRCh38_RefSeq2UCSC.txt', sep='\t', header=None)
chr_map.columns = ['seqid', 'chr']
chr_map[0:2]

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

,seqid,chr
0,NC_000001.11,chr1
1,NC_000002.12,chr2


In [43]:
df2 = df.merge(chr_map, on='seqid', how='left')
print (df2.shape, df.shape)
df[0:2]

(1514439, 10) (1514439, 9)


,tr_id,gene,seqid,type,chrom,start,stop,end,strand
0,NM_001005484.2,OR4F5,NC_000001.11,CDS,NC_000001.11,65565,65573,65573,+
1,NM_001005484.2,OR4F5,NC_000001.11,CDS,NC_000001.11,69037,70008,70008,+


In [44]:
refseq = df2[(df2['tr_id'].str.contains('NM_')) & (df2['chr'].isin(['chr'+str(i) for i in range(1, 23)]+['chrX', 'chrY']))][['chr', 'start', 'end', 'tr_id', 'strand', 'gene']]
refseq['tmp'] = 1
refseq[['chr', 'start', 'end', 'tr_id', 'tmp', 'strand']].to_csv('tmp_res/refseq_coding_exons_chr_NM.bed', sep='\t', index=False, header=None)

## Gencode 38 

May5, for some reason, there is gencode38 release in Hiller lab Genome browser mirror. Why not? 

In [45]:
bed_g38_CDS = pd.read_csv('data/Gencode38_HillerLab_table_browser_CDS.bed', sep='\t', header=None)
bed_g38_CDS[0:2]

,0,1,2,3,4,5
0,chr1,67093004,67093604,ENST00000684719.1_cds_0_0_chr1_67093005_r,0,-
1,chr1,67095234,67095421,ENST00000684719.1_cds_1_0_chr1_67095235_r,0,-


# Overlap between theoretical extensions & coding exons of gencode25

* bedtools sort -i tmp_res/cod_ex_25_corrected.bed > tmp_res/cod_ex_25_corrected_sorted.bed

* bedtools sort -i tmp_res/full_NTE.bed > tmp_res/full_NTE_sorted.bed

* bedtools intersect -a tmp_res/full_NTE_sorted.bed -b tmp_res/cod_ex_25_corrected_sorted.bed -wao -s > tmp_res/NTE_full_start_with_all_coding_exons_g25.bed

In [ ]:
* bedtools sort -i tmp_res/cod_ex_25_corrected.bed > tmp_res/cod_ex_25_corrected_sorted.bed

* bedtools sort -i tmp_res/full_NTE.bed > tmp_res/full_NTE_sorted.bed

* bedtools intersect -a tmp_res/full_NTE_sorted.bed -b tmp_res/cod_ex_25_corrected_sorted.bed -wao -s > tmp_res/NTE_full_start_with_all_coding_exons_g25.bed

### Parse it 

In [46]:
overlap_all_exons_g25 = pd.read_csv('tmp_res/NTE_full_start_with_all_coding_exons_g25.bed', sep='\t', header=None) 
overlap_all_exons_g25.columns = [0, 1, 2, 'tr_id_ext', 4, 5, 6, 7, 8, 'tr_id_cod', 10, 11, 'max_overlap_g25_all_exons']

# find sum overlap per pair of transcript with extension and transcript with coding exons 
#overlap_all_exons_g25_ = overlap_all_exons_g25[overlap_all_exons_g25[10] != -1]
overlap_all_exons_g25_sum = overlap_all_exons_g25.groupby(by=['tr_id_ext', 'tr_id_cod'])['max_overlap_g25_all_exons'].sum().reset_index()

# find max overall (regardkess of frame)
overlap_all_exons_g25_sum_max = overlap_all_exons_g25_sum.loc[overlap_all_exons_g25_sum.groupby('tr_id_ext')['max_overlap_g25_all_exons'].idxmax()]

print (overlap_all_exons_g25_sum_max.tr_id_ext.nunique(), overlap_all_exons_g25_sum_max.shape)

overlap_all_exons_g25_sum_max.columns = ['tr_id', 'tr_id_cod', 'max_sum_overlap_g25_all_exons']

#overlap_all_exons_g25_sum_max[0:3]

local_and_global_df_g25ovlp = local_and_global_df.merge(overlap_all_exons_g25_sum_max[['tr_id', 'max_sum_overlap_g25_all_exons']], on='tr_id', how='inner')

print (local_and_global_df_g25ovlp.shape, local_and_global_df.shape)

local_and_global_df_g25ovlp[0:5]

77820 (77820, 3)
(77820, 15) (77820, 14)


,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,strand,global_coo_primary,transcript_seq,5UTR_start_seq,max_sum_overlap_g25_all_exons
0,ENST00000624431.1,274,316,42,274,316,14.0,FO538757.2,274,chr1:182667-182708,+,chr1:182667-182708,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,0
1,ENST00000624735.1,0,236,236,2,236,78.0,FO538757.1,86,chr1:200087-200236,-,chr1:200087-200320,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,0
2,ENST00000420190.6,60,504,444,60,504,148.0,SAMD11,354,chr1:924282-924431,+,chr1:923988-924431,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,0
3,ENST00000437963.5,0,60,60,0,60,20.0,SAMD11,0,chr1:925150-925189+chr1:925922-925941,+,chr1:925150-925189+chr1:925922-925941,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,19
4,ENST00000342066.7,0,83,83,2,83,27.0,SAMD11,2,chr1:925740-925800+chr1:925922-925941,+,chr1:925740-925800+chr1:925922-925941,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,19


# Well, for the sake of DHFR gene for which there are ovlps with coding exons in another strand... 

* bedtools intersect -a tmp_res/full_NTE_sorted.bed -b tmp_res/cod_ex_25_corrected_sorted.bed -wao > tmp_res/NTE_full_start_with_all_coding_exons_g25_strandless.bed

In [47]:
overlap_all_exons_g25_s = pd.read_csv('tmp_res/NTE_full_start_with_all_coding_exons_g25_strandless.bed', sep='\t', header=None) 
overlap_all_exons_g25_s.columns = [0, 1, 2, 'tr_id_ext', 4, 5, 6, 7, 8, 'tr_id_cod', 10, 11, 'max_overlap_g25_all_exons_strandless']

# find sum overlap per pair of transcript with extension and transcript with coding exons 
#overlap_all_exons_g25_ = overlap_all_exons_g25[overlap_all_exons_g25[10] != -1]
overlap_all_exons_g25_sum_s = overlap_all_exons_g25_s.groupby(by=['tr_id_ext', 'tr_id_cod'])['max_overlap_g25_all_exons_strandless'].sum().reset_index()

# find max overall (regardkess of frame)
overlap_all_exons_g25_sum_max_s = overlap_all_exons_g25_sum_s.loc[overlap_all_exons_g25_sum_s.groupby('tr_id_ext')['max_overlap_g25_all_exons_strandless'].idxmax()]

print (overlap_all_exons_g25_sum_max_s.tr_id_ext.nunique(), overlap_all_exons_g25_sum_max_s.shape)

overlap_all_exons_g25_sum_max_s.columns = ['tr_id', 'tr_id_cod', 'max_overlap_g25_all_exons_strandless']

#overlap_all_exons_g25_sum_max[0:3]

local_and_global_df_g25ovlp_s = local_and_global_df_g25ovlp.merge(overlap_all_exons_g25_sum_max_s[['tr_id', 'max_overlap_g25_all_exons_strandless']], on='tr_id', how='inner')

print (local_and_global_df_g25ovlp_s.shape, local_and_global_df_g25ovlp_s.shape)

local_and_global_df_g25ovlp_s[0:5]

77820 (77820, 3)
(77820, 16) (77820, 16)


,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,strand,global_coo_primary,transcript_seq,5UTR_start_seq,max_sum_overlap_g25_all_exons,max_overlap_g25_all_exons_strandless
0,ENST00000624431.1,274,316,42,274,316,14.0,FO538757.2,274,chr1:182667-182708,+,chr1:182667-182708,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,0,0
1,ENST00000624735.1,0,236,236,2,236,78.0,FO538757.1,86,chr1:200087-200236,-,chr1:200087-200320,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,0,0
2,ENST00000420190.6,60,504,444,60,504,148.0,SAMD11,354,chr1:924282-924431,+,chr1:923988-924431,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,0,0
3,ENST00000437963.5,0,60,60,0,60,20.0,SAMD11,0,chr1:925150-925189+chr1:925922-925941,+,chr1:925150-925189+chr1:925922-925941,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,19,19
4,ENST00000342066.7,0,83,83,2,83,27.0,SAMD11,2,chr1:925740-925800+chr1:925922-925941,+,chr1:925740-925800+chr1:925922-925941,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,19,19


In [48]:
local_and_global_df_g25ovlp_s[local_and_global_df_g25ovlp_s['tr_id'] == 'ENST00000439211.6']

,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,strand,global_coo_primary,transcript_seq,5UTR_start_seq,max_sum_overlap_g25_all_exons,max_overlap_g25_all_exons_strandless
20599,ENST00000439211.6,0,494,494,2,494,164.0,DHFR,344,chr5:80654490-80654639,-,chr5:80654490-80654981,AGTCCCAGACAGAACCTACTATGTGCGGCGGCAGCTGGGGCGGGAA...,AGTCCCAGACAGAACCTACTATGTGCGGCGGCAGCTGGGGCGGGAA...,0,237


# Overlap between theoretical extensions & coding exons of gencode35

bedtools sort -i tmp_res/cod_ex_35_corrected.bed > tmp_res/cod_ex_35_corrected_sorted.bed

bedtools sort -i tmp_res/full_NTE.bed > tmp_res/full_NTE_sorted.bed

bedtools intersect -a tmp_res/full_NTE_sorted.bed -b tmp_res/cod_ex_35_corrected_sorted.bed -wao -s > tmp_res/NTE_full_start_with_all_coding_exons_g35.bed

### parse it 

In [55]:
overlap_all_exons_g35 = pd.read_csv('tmp_res/NTE_full_start_with_all_coding_exons_g35.bed', sep='\t', header=None) 
overlap_all_exons_g35.columns = [0, 1, 2, 'tr_id_ext', 4, 5, 6, 7, 8, 'tr_id_cod', 10, 11, 'max_overlap_g35_all_exons']

# find sum overlap per pair of transcript with extension and transcript with coding exons 
#overlap_all_exons_g25_ = overlap_all_exons_g25[overlap_all_exons_g25[10] != -1]
overlap_all_exons_g35_sum = overlap_all_exons_g35.groupby(by=['tr_id_ext', 'tr_id_cod'])['max_overlap_g35_all_exons'].sum().reset_index()

# find max overall (regardkess of frame)
overlap_all_exons_g35_sum_max = overlap_all_exons_g35_sum.loc[overlap_all_exons_g35_sum.groupby('tr_id_ext')['max_overlap_g35_all_exons'].idxmax()]

print (overlap_all_exons_g35_sum_max.tr_id_ext.nunique(), overlap_all_exons_g35_sum_max.shape)

overlap_all_exons_g35_sum_max.columns = ['tr_id', 'tr_id_cod', 'max_sum_overlap_g35_all_exons']

#overlap_all_exons_g35_sum_max[0:3]

local_and_global_df_g35ovlp = local_and_global_df_g25ovlp_s.merge(overlap_all_exons_g35_sum_max[['tr_id', 'max_sum_overlap_g35_all_exons']], on='tr_id', how='inner')

print (local_and_global_df_g35ovlp.shape, local_and_global_df.shape)

local_and_global_df_g35ovlp[0:5]

77820 (77820, 3)
(77820, 17) (77820, 14)


,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,strand,global_coo_primary,transcript_seq,5UTR_start_seq,max_sum_overlap_g25_all_exons,max_overlap_g25_all_exons_strandless,max_sum_overlap_g35_all_exons
0,ENST00000624431.1,274,316,42,274,316,14.0,FO538757.2,274,chr1:182667-182708,+,chr1:182667-182708,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,0,0,0
1,ENST00000624735.1,0,236,236,2,236,78.0,FO538757.1,86,chr1:200087-200236,-,chr1:200087-200320,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,0,0,0
2,ENST00000420190.6,60,504,444,60,504,148.0,SAMD11,354,chr1:924282-924431,+,chr1:923988-924431,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,0,0,0
3,ENST00000437963.5,0,60,60,0,60,20.0,SAMD11,0,chr1:925150-925189+chr1:925922-925941,+,chr1:925150-925189+chr1:925922-925941,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,19,19,19
4,ENST00000342066.7,0,83,83,2,83,27.0,SAMD11,2,chr1:925740-925800+chr1:925922-925941,+,chr1:925740-925800+chr1:925922-925941,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,19,19,19


# gencode35 strandless 

bedtools intersect -a tmp_res/full_NTE_sorted.bed -b tmp_res/cod_ex_35_corrected_sorted.bed -wao > tmp_res/NTE_full_start_with_all_coding_exons_g35_strandless.bed

In [71]:
overlap_all_exons_g35_s = pd.read_csv('tmp_res/NTE_full_start_with_all_coding_exons_g35_strandless.bed', sep='\t', header=None) 
overlap_all_exons_g35_s.columns = [0, 1, 2, 'tr_id_ext', 4, 5, 6, 7, 8, 'tr_id_cod', 10, 11, 
                                   'max_overlap_g35_all_exons_strandless']

# find sum overlap per pair of transcript with extension and transcript with coding exons 
#overlap_all_exons_g25_ = overlap_all_exons_g25[overlap_all_exons_g25[10] != -1]
overlap_all_exons_g35_sum_s = overlap_all_exons_g35_s.groupby(by=['tr_id_ext', 'tr_id_cod'])['max_overlap_g35_all_exons_strandless'].sum().reset_index()

# find max overall (regardkess of frame)
overlap_all_exons_g35_sum_max_s = overlap_all_exons_g35_sum_s.loc[overlap_all_exons_g35_sum_s.groupby('tr_id_ext')['max_overlap_g35_all_exons_strandless'].idxmax()]

print (overlap_all_exons_g35_sum_max_s.tr_id_ext.nunique(), overlap_all_exons_g35_sum_max_s.shape)

overlap_all_exons_g35_sum_max_s.columns = ['tr_id', 'tr_id_cod', 'max_sum_overlap_g35_all_exons_strandless']

#overlap_all_exons_g35_sum_max[0:3]

local_and_global_df_g35ovlp_s = local_and_global_df_g35ovlp.merge(overlap_all_exons_g35_sum_max_s[['tr_id', 'max_sum_overlap_g35_all_exons_strandless']], on='tr_id', how='inner')

print (local_and_global_df_g35ovlp_s.shape, local_and_global_df.shape)

local_and_global_df_g35ovlp_s[0:5]

77820 (77820, 3)
(77820, 18) (77820, 14)


,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,strand,global_coo_primary,transcript_seq,5UTR_start_seq,max_sum_overlap_g25_all_exons,max_overlap_g25_all_exons_strandless,max_sum_overlap_g35_all_exons,max_sum_overlap_g35_all_exons_strandless
0,ENST00000624431.1,274,316,42,274,316,14.0,FO538757.2,274,chr1:182667-182708,+,chr1:182667-182708,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,0,0,0,0
1,ENST00000624735.1,0,236,236,2,236,78.0,FO538757.1,86,chr1:200087-200236,-,chr1:200087-200320,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,0,0,0,0
2,ENST00000420190.6,60,504,444,60,504,148.0,SAMD11,354,chr1:924282-924431,+,chr1:923988-924431,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,0,0,0,0
3,ENST00000437963.5,0,60,60,0,60,20.0,SAMD11,0,chr1:925150-925189+chr1:925922-925941,+,chr1:925150-925189+chr1:925922-925941,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,19,19,19,19
4,ENST00000342066.7,0,83,83,2,83,27.0,SAMD11,2,chr1:925740-925800+chr1:925922-925941,+,chr1:925740-925800+chr1:925922-925941,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,19,19,19,19


# Overlap between theoretical extensions & coding exons of gencode38

bedtools sort -i data/Gencode38_HillerLab_table_browser_CDS.bed > data/Gencode38_HillerLab_table_browser_CDS_sorted.bed

bedtools sort -i tmp_res/full_NTE.bed > tmp_res/full_NTE_sorted.bed

bedtools intersect -a tmp_res/full_NTE_sorted.bed -b data/Gencode38_HillerLab_table_browser_CDS_sorted.bed -wao -s > tmp_res/NTE_full_start_with_all_coding_exons_g38.bed

### parse it

In [72]:
overlap_all_exons_g38 = pd.read_csv('tmp_res/NTE_full_start_with_all_coding_exons_g38.bed', sep='\t', header=None) 
overlap_all_exons_g38.columns = [0, 1, 2, 'tr_id_ext', 4, 5, 6, 7, 8, 'tr_id_cod', 10, 11, 'max_overlap_g38_all_exons']

# find sum overlap per pair of transcript with extension and transcript with coding exons 
#overlap_all_exons_g25_ = overlap_all_exons_g25[overlap_all_exons_g25[10] != -1]
overlap_all_exons_g38_sum = overlap_all_exons_g38.groupby(by=['tr_id_ext', 'tr_id_cod'])['max_overlap_g38_all_exons'].sum().reset_index()

# find max overall (regardkess of frame)
overlap_all_exons_g38_sum_max = overlap_all_exons_g38_sum.loc[overlap_all_exons_g38_sum.groupby('tr_id_ext')['max_overlap_g38_all_exons'].idxmax()]

print (overlap_all_exons_g38_sum_max.tr_id_ext.nunique(), overlap_all_exons_g38_sum_max.shape)

overlap_all_exons_g38_sum_max.columns = ['tr_id', 'tr_id_cod', 'max_sum_overlap_g38_all_exons']

#overlap_all_exons_g35_sum_max[0:3]

local_and_global_df_g38ovlp = local_and_global_df_g35ovlp_s.merge(overlap_all_exons_g38_sum_max[['tr_id', 'max_sum_overlap_g38_all_exons']], on='tr_id', how='inner')

print (local_and_global_df_g38ovlp.shape, local_and_global_df.shape)

local_and_global_df_g38ovlp[0:5]

77820 (77820, 3)
(77820, 19) (77820, 14)


,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,strand,global_coo_primary,transcript_seq,5UTR_start_seq,max_sum_overlap_g25_all_exons,max_overlap_g25_all_exons_strandless,max_sum_overlap_g35_all_exons,max_sum_overlap_g35_all_exons_strandless,max_sum_overlap_g38_all_exons
0,ENST00000624431.1,274,316,42,274,316,14.0,FO538757.2,274,chr1:182667-182708,+,chr1:182667-182708,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,0,0,0,0,0
1,ENST00000624735.1,0,236,236,2,236,78.0,FO538757.1,86,chr1:200087-200236,-,chr1:200087-200320,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,0,0,0,0,0
2,ENST00000420190.6,60,504,444,60,504,148.0,SAMD11,354,chr1:924282-924431,+,chr1:923988-924431,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,0,0,0,0,0
3,ENST00000437963.5,0,60,60,0,60,20.0,SAMD11,0,chr1:925150-925189+chr1:925922-925941,+,chr1:925150-925189+chr1:925922-925941,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,19,19,19,19,19
4,ENST00000342066.7,0,83,83,2,83,27.0,SAMD11,2,chr1:925740-925800+chr1:925922-925941,+,chr1:925740-925800+chr1:925922-925941,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,19,19,19,19,19


# Strandless gencode38

bedtools intersect -a tmp_res/full_NTE_sorted.bed -b data/Gencode38_HillerLab_table_browser_CDS_sorted.bed -wao > tmp_res/NTE_full_start_with_all_coding_exons_g38_strandless.bed

In [73]:
overlap_all_exons_g38_s = pd.read_csv('tmp_res/NTE_full_start_with_all_coding_exons_g38_strandless.bed', sep='\t', header=None) 
overlap_all_exons_g38_s.columns = [0, 1, 2, 'tr_id_ext', 4, 5, 6, 7, 8, 'tr_id_cod', 
                                 10, 11, 'max_overlap_g38_all_exons_strandless']

# find sum overlap per pair of transcript with extension and transcript with coding exons 
#overlap_all_exons_g25_ = overlap_all_exons_g25[overlap_all_exons_g25[10] != -1]
overlap_all_exons_g38_sum_s = overlap_all_exons_g38_s.groupby(by=['tr_id_ext', 'tr_id_cod'])['max_overlap_g38_all_exons_strandless'].sum().reset_index()

# find max overall (regardkess of frame)
overlap_all_exons_g38_sum_max_s = overlap_all_exons_g38_sum_s.loc[overlap_all_exons_g38_sum_s.groupby('tr_id_ext')['max_overlap_g38_all_exons_strandless'].idxmax()]

print (overlap_all_exons_g38_sum_max_s.tr_id_ext.nunique(), overlap_all_exons_g38_sum_max_s.shape)

overlap_all_exons_g38_sum_max_s.columns = ['tr_id', 'tr_id_cod', 'max_sum_overlap_g38_all_exons_strandless']

#overlap_all_exons_g35_sum_max[0:3]

local_and_global_df_g38ovlp_s = local_and_global_df_g38ovlp.merge(overlap_all_exons_g38_sum_max_s[['tr_id', 'max_sum_overlap_g38_all_exons_strandless']], on='tr_id', how='inner')

print (local_and_global_df_g38ovlp_s.shape, local_and_global_df.shape)

local_and_global_df_g38ovlp_s[0:5]

77820 (77820, 3)
(77820, 20) (77820, 14)


,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,strand,global_coo_primary,transcript_seq,5UTR_start_seq,max_sum_overlap_g25_all_exons,max_overlap_g25_all_exons_strandless,max_sum_overlap_g35_all_exons,max_sum_overlap_g35_all_exons_strandless,max_sum_overlap_g38_all_exons,max_sum_overlap_g38_all_exons_strandless
0,ENST00000624431.1,274,316,42,274,316,14.0,FO538757.2,274,chr1:182667-182708,+,chr1:182667-182708,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,0,0,0,0,0,0
1,ENST00000624735.1,0,236,236,2,236,78.0,FO538757.1,86,chr1:200087-200236,-,chr1:200087-200320,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,0,0,0,0,0,0
2,ENST00000420190.6,60,504,444,60,504,148.0,SAMD11,354,chr1:924282-924431,+,chr1:923988-924431,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,0,0,0,0,0,0
3,ENST00000437963.5,0,60,60,0,60,20.0,SAMD11,0,chr1:925150-925189+chr1:925922-925941,+,chr1:925150-925189+chr1:925922-925941,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,19,19,19,19,19,19
4,ENST00000342066.7,0,83,83,2,83,27.0,SAMD11,2,chr1:925740-925800+chr1:925922-925941,+,chr1:925740-925800+chr1:925922-925941,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,19,19,19,19,19,19


# Overlap between theoretical extensions & coding exons of refseq

In [68]:
refseq_cod_ex = pd.read_csv('tmp_res/refseq_coding_exons_chr_NM.bed', sep='\t', header=None)

refseq_cod_ex.columns = ['chr_cod', 'start_cod', 'end_cod', 'tr_id_cod', 'exon_cod', 'strand_cod']

refseq_cod_ex[0:2]

,chr_cod,start_cod,end_cod,tr_id_cod,exon_cod,strand_cod
0,chr1,65565,65573,NM_001005484.2,1,+
1,chr1,69037,70008,NM_001005484.2,1,+


In [69]:
res1 = dict(tuple(refseq_cod_ex.groupby('tr_id_cod')))

li = []
for k,v in res1.items():
    if v.iloc[0].strand_cod == '+':
        v1 = v.sort_values(by='start_cod', ascending=True)
        v1['exon_cod'] = [i for i in range(1, v1.shape[0]+1)]
    else:
        v1 = v.sort_values(by='start_cod', ascending=False)
        v1['exon_cod'] = [i for i in range(1, v1.shape[0]+1)]
    li.append(v1)
    
refseq_cod_ex = pd.concat(li)

refseq_cod_ex['start_cod'] = refseq_cod_ex['start_cod'] - 1

refseq_cod_ex = refseq_cod_ex.sort_values(by=['tr_id_cod', 'exon_cod'])

refseq_cod_ex.to_csv('tmp_res/RefSeq_cod_ex_corrected_start.bed', sep='\t', index=False, header=None)

bedtools sort -i tmp_res/RefSeq_cod_ex_corrected_start.bed > tmp_res/RefSeq_cod_ex_corrected_start_sorted.bed

bedtools sort -i tmp_res/full_NTE.bed > tmp_res/full_NTE_sorted.bed

bedtools intersect -a tmp_res/full_NTE_sorted.bed -b tmp_res/RefSeq_cod_ex_corrected_start_sorted.bed -wao -s > tmp_res/NTE_full_start_with_all_coding_exons_refseq.bed

In [74]:
overlap_all_exons_refseq = pd.read_csv('tmp_res/NTE_full_start_with_all_coding_exons_refseq.bed', sep='\t', header=None) 
overlap_all_exons_refseq.columns = [0, 1, 2, 'tr_id_ext', 4, 5, 6, 7, 8, 'tr_id_cod', 10, 11, 'max_overlap_refseq_all_exons']

# find sum overlap per pair of transcript with extension and transcript with coding exons 
#overlap_all_exons_g25_ = overlap_all_exons_g25[overlap_all_exons_g25[10] != -1]
overlap_all_exons_refseq_sum = overlap_all_exons_refseq.groupby(by=['tr_id_ext', 'tr_id_cod'])['max_overlap_refseq_all_exons'].sum().reset_index()

# find max overall (regardkess of frame)
overlap_all_exons_refseq_sum_max = overlap_all_exons_refseq_sum.loc[overlap_all_exons_refseq_sum.groupby('tr_id_ext')['max_overlap_refseq_all_exons'].idxmax()]

print (overlap_all_exons_refseq_sum_max.tr_id_ext.nunique(), overlap_all_exons_refseq_sum_max.shape)

overlap_all_exons_refseq_sum_max.columns = ['tr_id', 'tr_id_cod', 'max_sum_overlap_refseq_all_exons']

#overlap_all_exons_refseq_sum_max[0:3]

local_and_global_df_all_ovlp_refseq = local_and_global_df_g38ovlp_s.merge(overlap_all_exons_refseq_sum_max[['tr_id', 'max_sum_overlap_refseq_all_exons']], on='tr_id', how='inner')

print (local_and_global_df_all_ovlp_refseq.shape, local_and_global_df.shape)

local_and_global_df_all_ovlp_refseq[0:5]

77820 (77820, 3)
(77820, 21) (77820, 14)


,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,...,global_coo_primary,transcript_seq,5UTR_start_seq,max_sum_overlap_g25_all_exons,max_overlap_g25_all_exons_strandless,max_sum_overlap_g35_all_exons,max_sum_overlap_g35_all_exons_strandless,max_sum_overlap_g38_all_exons,max_sum_overlap_g38_all_exons_strandless,max_sum_overlap_refseq_all_exons
0,ENST00000624431.1,274,316,42,274,316,14.0,FO538757.2,274,chr1:182667-182708,...,chr1:182667-182708,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,0,0,0,0,0,0,0
1,ENST00000624735.1,0,236,236,2,236,78.0,FO538757.1,86,chr1:200087-200236,...,chr1:200087-200320,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,0,0,0,0,0,0,0
2,ENST00000420190.6,60,504,444,60,504,148.0,SAMD11,354,chr1:924282-924431,...,chr1:923988-924431,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,0,0,0,0,0,0,0
3,ENST00000437963.5,0,60,60,0,60,20.0,SAMD11,0,chr1:925150-925189+chr1:925922-925941,...,chr1:925150-925189+chr1:925922-925941,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,19,19,19,19,19,19,19
4,ENST00000342066.7,0,83,83,2,83,27.0,SAMD11,2,chr1:925740-925800+chr1:925922-925941,...,chr1:925740-925800+chr1:925922-925941,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,19,19,19,19,19,19,19


# Strandless Refseq

bedtools intersect -a tmp_res/full_NTE_sorted.bed -b tmp_res/RefSeq_cod_ex_corrected_start_sorted.bed -wao > tmp_res/NTE_full_start_with_all_coding_exons_refseq_strandless.bed

In [89]:
overlap_all_exons_refseq_s = pd.read_csv('tmp_res/NTE_full_start_with_all_coding_exons_refseq_strandless.bed', sep='\t', header=None) 
overlap_all_exons_refseq_s.columns = [0, 1, 2, 'tr_id_ext', 4, 5, 6, 7, 8, 
                                    'tr_id_cod', 10, 11, 'max_overlap_refseq_all_exons_strandless']

# find sum overlap per pair of transcript with extension and transcript with coding exons 
#overlap_all_exons_g25_ = overlap_all_exons_g25[overlap_all_exons_g25[10] != -1]
overlap_all_exons_refseq_sum_s = overlap_all_exons_refseq_s.groupby(by=['tr_id_ext', 'tr_id_cod'])['max_overlap_refseq_all_exons_strandless'].sum().reset_index()

# find max overall (regardkess of frame)
overlap_all_exons_refseq_sum_max_s = overlap_all_exons_refseq_sum_s.loc[overlap_all_exons_refseq_sum_s.groupby('tr_id_ext')['max_overlap_refseq_all_exons_strandless'].idxmax()]

print (overlap_all_exons_refseq_sum_max_s.tr_id_ext.nunique(), overlap_all_exons_refseq_sum_max_s.shape)

overlap_all_exons_refseq_sum_max_s.columns = ['tr_id', 'tr_id_cod', 'max_sum_overlap_refseq_all_exons_strandless']

#overlap_all_exons_refseq_sum_max[0:3]

local_and_global_df_all_ovlp = local_and_global_df_all_ovlp_refseq.merge(overlap_all_exons_refseq_sum_max_s[['tr_id', 'max_sum_overlap_refseq_all_exons_strandless']], on='tr_id', how='inner')

print (local_and_global_df_all_ovlp.shape, local_and_global_df.shape)

local_and_global_df_all_ovlp[0:5]

77820 (77820, 3)
(77820, 22) (77820, 14)


,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,...,transcript_seq,5UTR_start_seq,max_sum_overlap_g25_all_exons,max_overlap_g25_all_exons_strandless,max_sum_overlap_g35_all_exons,max_sum_overlap_g35_all_exons_strandless,max_sum_overlap_g38_all_exons,max_sum_overlap_g38_all_exons_strandless,max_sum_overlap_refseq_all_exons,max_sum_overlap_refseq_all_exons_strandless
0,ENST00000624431.1,274,316,42,274,316,14.0,FO538757.2,274,chr1:182667-182708,...,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,0,0,0,0,0,0,0,0
1,ENST00000624735.1,0,236,236,2,236,78.0,FO538757.1,86,chr1:200087-200236,...,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,GGAAAGCGGGTCAAGGCGTAGGGCTGGAGGGCAGGGGCGGGCCCTG...,0,0,0,0,0,0,0,0
2,ENST00000420190.6,60,504,444,60,504,148.0,SAMD11,354,chr1:924282-924431,...,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,CGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTGGACGC...,0,0,0,0,0,0,0,0
3,ENST00000437963.5,0,60,60,0,60,20.0,SAMD11,0,chr1:925150-925189+chr1:925922-925941,...,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,CAGCGCTTGGGGCTCGCGGGCCGCTCCCTCCGCTCGGAAGGGAAAA...,19,19,19,19,19,19,19,19
4,ENST00000342066.7,0,83,83,2,83,27.0,SAMD11,2,chr1:925740-925800+chr1:925922-925941,...,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,CCAGCAGATCCCTGCGGCGTTCGCGAGGGTGGGACGGGAAGCGGGC...,19,19,19,19,19,19,19,19


In [90]:
# ENST00000439211.6

In [91]:
overlap_all_exons_refseq_s[overlap_all_exons_refseq_s['tr_id_ext'] == 'ENST00000439211.6']

,0,1,2,tr_id_ext,4,5,6,7,8,tr_id_cod,10,11,max_overlap_refseq_all_exons_strandless
133334,chr5,80654490,80654981,ENST00000439211.6,1,-,chr5,80654727,80654964,NM_002439.5,1,+,237


In [92]:
overlap_all_exons_refseq_s[overlap_all_exons_refseq_s['tr_id_cod'] == '']

,0,1,2,tr_id_ext,4,5,6,7,8,tr_id_cod,10,11,max_overlap_refseq_all_exons_strandless


# Combine overlaps into 1 column

In [93]:
local_and_global_df_all_ovlp['ovlp'] = local_and_global_df_all_ovlp['max_sum_overlap_g25_all_exons'].astype(str) + ';' + local_and_global_df_all_ovlp['max_sum_overlap_g35_all_exons'].astype(str) + ';' + local_and_global_df_all_ovlp['max_sum_overlap_refseq_all_exons'].astype(str)

local_and_global_df_all_ovlp['ovlp_strandless'] = local_and_global_df_all_ovlp['max_overlap_g25_all_exons_strandless'].astype(str) + ';' + local_and_global_df_all_ovlp['max_sum_overlap_g35_all_exons_strandless'].astype(str) + ';' + local_and_global_df_all_ovlp['max_sum_overlap_refseq_all_exons_strandless'].astype(str)

local_and_global_df_all_ovlp['ovlp2'] = local_and_global_df_all_ovlp['max_sum_overlap_g25_all_exons'].astype(str) + ';' + local_and_global_df_all_ovlp['max_sum_overlap_g35_all_exons'].astype(str) + ';' + local_and_global_df_all_ovlp['max_sum_overlap_g38_all_exons'].astype(str) +';'+local_and_global_df_all_ovlp['max_sum_overlap_refseq_all_exons'].astype(str)

local_and_global_df_all_ovlp['ovlp2_strandless'] = local_and_global_df_all_ovlp['max_overlap_g25_all_exons_strandless'].astype(str) + ';' + local_and_global_df_all_ovlp['max_sum_overlap_g35_all_exons_strandless'].astype(str) + ';' + local_and_global_df_all_ovlp['max_sum_overlap_g38_all_exons_strandless'].astype(str) +';'+local_and_global_df_all_ovlp['max_sum_overlap_refseq_all_exons_strandless'].astype(str)


local_and_global_df_all_ovlp[['tr_id', 'ovlp', 'ovlp2', 'ovlp_strandless', 'ovlp2_strandless']][0:5]

,tr_id,ovlp,ovlp2,ovlp_strandless,ovlp2_strandless
0,ENST00000624431.1,0;0;0,0;0;0;0,0;0;0,0;0;0;0
1,ENST00000624735.1,0;0;0,0;0;0;0,0;0;0,0;0;0;0
2,ENST00000420190.6,0;0;0,0;0;0;0,0;0;0,0;0;0;0
3,ENST00000437963.5,19;19;19,19;19;19;19,19;19;19,19;19;19;19
4,ENST00000342066.7,19;19;19,19;19;19;19,19;19;19,19;19;19;19


In [94]:
# # g25 - g35 - g38 - refseq
# 5'UTR of DHFR gene has overlap with coding exons of MSH3 in a different strand

local_and_global_df_all_ovlp[local_and_global_df_all_ovlp['tr_id'] == 'ENST00000439211.6']

,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,...,max_sum_overlap_g35_all_exons,max_sum_overlap_g35_all_exons_strandless,max_sum_overlap_g38_all_exons,max_sum_overlap_g38_all_exons_strandless,max_sum_overlap_refseq_all_exons,max_sum_overlap_refseq_all_exons_strandless,ovlp,ovlp_strandless,ovlp2,ovlp2_strandless
20599,ENST00000439211.6,0,494,494,2,494,164.0,DHFR,344,chr5:80654490-80654639,...,0,237,0,237,0,237,0;0;0,237;237;237,0;0;0;0,237;237;237;237


# Add PhyloCSF score results 

* note: if length of extension is <=1 codon - no PhyloCSF score was calculated

In [96]:
li = []

for path in glob.glob('data/PhyloCSF_50_NTE/*/*.txt'):
    try: 
        tmp = pd.read_csv(path, sep='\t', header=None)
        li.append(tmp)
        
    except Exception as e:     
        with open(path, 'r') as f:
            li1 = []
            for l in f.readlines():
                if (l.find('Failure("invalid MFA alignment: Invalid_argument(\\"Option.get\\")")') == -1) | (l.find('Failure("no regions successfully evaluated")') == -1):
                    li1.append(l.strip().split('\t'))
                else:
                    print (l.split('\t')[0])
            tmp2 = pd.DataFrame(li1)
            li.append(tmp2)

PhyloCSF_50 = pd.concat(li)[[0, 1, 2]]

PhyloCSF_50['tr_id'] = [x.split('/')[-1].split('.fasta')[0] for x in PhyloCSF_50[0].tolist()]

PhyloCSF_50 = PhyloCSF_50[['tr_id', 2]]

PhyloCSF_50 = PhyloCSF_50[(PhyloCSF_50[2] != 'Failure("no regions successfully evaluated")') & (PhyloCSF_50[2] != 'Failure("invalid MFA alignment: Invalid_argument(\\"Option.get\\")")')]

PhyloCSF_50[2] = PhyloCSF_50[2].astype(float)

PhyloCSF_50.columns = ['tr_id', 'PhyloCSF120score']

PhyloCSF_50['tr_id1'] = [x.split('.')[0] for x in PhyloCSF_50['tr_id'].tolist()]

local_and_global_df_all_ovlp_PhyloCSF = local_and_global_df_all_ovlp.merge(PhyloCSF_50, on='tr_id', how='inner')

print (local_and_global_df_all_ovlp_PhyloCSF.shape, local_and_global_df_all_ovlp.shape)

(77513, 28) (77820, 26)


In [97]:
# local_and_global_df_all_ovlp_PhyloCSF.to_csv('tmp_res/local_and_global_df_g25ovlp_g35ovlp_refseqovlp_FRAMES_PhyloCSF.txt', sep='\t', index=False)

# Add number of records for PhyloCSF

* file number_of_records_in_alignments_PCSF.txt was calculated on a server and transfered locally

In [98]:
size_of_al = pd.read_csv('tmp_res/number_of_records_in_alignments_PCSF.txt', sep='\t')

size_of_al['tr_id'] = size_of_al['name'].str.split('.fasta').str[0]

local_and_global_df_all_ovlp_PhyloCSF_rec = local_and_global_df_all_ovlp_PhyloCSF.merge(size_of_al[['tr_id', 'number_of_records', 'records_names']], how='left')

print (local_and_global_df_all_ovlp_PhyloCSF.shape, local_and_global_df_all_ovlp_PhyloCSF_rec.shape)

print (local_and_global_df_all_ovlp_PhyloCSF_rec[local_and_global_df_all_ovlp_PhyloCSF_rec['number_of_records'].isna()].tr_id.nunique())

(77513, 28) (77513, 30)
0


In [100]:
local_and_global_df_all_ovlp_PhyloCSF_rec.to_csv('tmp_res/local_and_global_df_g25ovlp_g35ovlp_g38ovlp_refseqovlp_strand_FRAMES_PhyloCSF.txt', sep='\t', index=False)